In [2]:
import warnings
warnings.filterwarnings("ignore",category=FutureWarning)

import scanpy as sc
import numpy as np
from scipy import sparse
import random
import numpy as np
import scipy.sparse as sp

## Create new dataset

In [42]:
# PARAMETERS FOR THE SUBSET
data_path = "/cluster/work/bewi/data/tahoe100/h5ad/controls_Trametinib_mergedtahoe100_hvg.h5ad"
adata = sc.read_h5ad(data_path)
export_path = "/cluster/work/bewi/members/rquiles/experiments/datasets/test_3cell_lines.h5ad"

change_col_names = {"drug":"Agg_Treatment", "cell_line": "covariates"}
control_name = "DMSO_TF"
cell_lines_keep = np.unique(adata.obs["cell_line"])[:3]
treatments_keep = np.unique(adata.obs["drugname_drugconc"])[[0,1,2]]

In [43]:
## UPDATE COLUMNS NEW DATASET
adata.obs = adata.obs.rename(columns=change_col_names)
adata.obs["control"] = adata.obs[change_col_names["drug"]] == control_name
adata.obs["control"] = adata.obs["control"].astype(int)
adata.uns["fields"] = []
adata.obs["dose"] = adata.obs["drugname_drugconc"].str.split(",").str[1].astype(float)

In [44]:
## SUBSET THE DATASET
idx=[] #List of row indexes to mantain in the AnnData
keep_rows = []

keep_rows = []
for cell_line in cell_lines_keep:
    for treatment in treatments_keep:
        mask = (adata.obs["covariates"] == cell_line) & (adata.obs["drugname_drugconc"] == treatment)
        row_indexes = adata.obs[mask].index
        keep_rows.extend(row_indexes)
        
# Randomize row order
random.shuffle(keep_rows)

#Subset the AnnData
filtered_adata = adata[keep_rows, :]

In [45]:
## PREPROCESS AND EXPORT
sc.pp.normalize_total(filtered_adata, target_sum=1e4)
sc.pp.log1p(filtered_adata)
sc.pp.highly_variable_genes(filtered_adata, n_top_genes=2000, subset=True)
sc.pp.scale(filtered_adata, max_value=10)
filtered_adata.write(export_path)

/cluster/home/rquiles/miniconda3/envs/fcr-env/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


## Test

In [4]:
import_path = "/cluster/work/bewi/members/rquiles/experiments/datasets/test_3cell_lines.h5ad"
adata = sc.read_h5ad(import_path)
adata.obs

,sample,gene_count,tscp_count,mread_count,drugname_drugconc,Agg_Treatment,covariates,sublibrary,BARCODE,pcnt_mito,S_score,G2M_score,phase,pass_filter,cell_name,plate,source_plate,n_genes,control,dose
BARCODE_SUB_LIB_ID,,,,,,,,,,,,,,,,,,,,
95_112_090-lib_2326,smp_1685,758,1018,1198,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0023,lib_2326,95_112_090,0.141454,-0.067460,-0.009385,G1,full,A549,plate2,plate2,758,1,0.00
96_152_125-lib_1504,smp_2262,2494,3849,4494,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0069,lib_1504,96_152_125,0.112497,-1.131693,-0.955311,G1,full,SK-MEL-2,plate8,plate8,2494,1,0.00
96_052_115-lib_1793,smp_2550,977,1431,1689,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0023,lib_1793,96_052_115,0.097834,-0.062429,0.048583,G2M,full,A549,plate11,plate11,977,1,0.00
26_012_009-lib_1473,smp_2096,643,809,973,"[('Trametinib (DMSO_TF solvate)', 0.05, 'uM')]",Trametinib (DMSO_TF solvate),CVCL_0069,lib_1473,26_012_009,0.044499,-0.066986,-0.048077,G1,full,SK-MEL-2,plate7,plate7,643,0,0.05
95_145_002-lib_1729,smp_2453,949,1268,1438,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0023,lib_1729,95_145_002,0.093060,-0.268057,-0.290476,G1,full,A549,plate10,plate10,949,1,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96_161_035-lib_1612,smp_2358,714,926,1066,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0023,lib_1612,96_161_035,0.073434,-0.033333,0.173353,G2M,full,A549,plate9,plate9,714,1,0.00
95_031_030-lib_1791,smp_2549,1105,1621,1908,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0023,lib_1791,95_031_030,0.068476,-0.076190,0.030037,G2M,full,A549,plate11,plate11,1105,1,0.00
95_160_068-lib_1589,smp_2357,932,1372,1639,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0023,lib_1589,95_160_068,0.083819,-0.066986,-0.028524,G1,full,A549,plate9,plate9,932,1,0.00


In [6]:
if "sample" in adata.obs:
    print("sample exists")

ZXs exists
